In [5]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'
!kaggle competitions download -c dogs-vs-cats
!unzip -q train.zip -d . 

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 60.0MB/s]
 97% 528M/543M [00:03<00:00, 152MB/s]
100% 543M/543M [00:03<00:00, 164MB/s]
 96% 260M/271M [00:02<00:00, 123MB/s] 
100% 271M/271M [00:02<00:00, 130MB/s]


In [7]:
import os 
import tensorflow as tf 
import shutil

#os.mkdir('/content/dataset')
#os.mkdir('/content/dataset/cat')
#os.mkdir('/content/dataset/dog')

for i in os.listdir('/content/train/'):
  if 'cat' in i:
    shutil.copyfile( '/content/train/' + i , '/content/dataset/cat/' + i )
  if 'dog' in i:
    shutil.copyfile( '/content/train/' + i , '/content/dataset/dog/' + i )

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/dataset/',
  image_size=(150,150),
  batch_size=64,
  subset='training',
  validation_split=0.2,
  seed=1234
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/dataset/',
  image_size=(150,150),
  batch_size=64,
  subset='validation',
  validation_split=0.2,
  seed=1234
)

print(train_ds)

def 전처리함수(i, 정답):
  i = tf.cast( i/255.0, tf.float32 )
  return i, 정답

train_ds = train_ds.map(전처리함수)
val_ds = val_ds.map(전처리함수)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<BatchDataset shapes: ((None, 150, 150, 3), (None,)), types: (tf.float32, tf.int32)>


구글의 inception_v3의 weight 파일을 가져다 씁니다.

In [8]:
import requests

url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
r = requests.get(url, allow_redirects=True)

open('inception_v3.h5', 'wb').write(r.content)

87910968

include_top은 마지막 출력층입니다.(마지막 Dense 레이어)

In [12]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

inception_model=InceptionV3(input_shape=(150,150,3),include_top=False,weights=None)
inception_model.load_weights('inception_v3.h5')

inception_model.summary()

for i in inception_model.layers:
  i.trainable=False #이 레이어는 학습하지 말아달라는 뜻입니다.
''' 만약 mixed6부터 트레이닝을 조금하고 싶다면 (fine tuning)
unfreeze=False
for i in inception_model.layer:
  if i.name=='mixed6':
    unfreeze=True
  if unfreeze==True:
    i.trainable= True
'''
final_layer=inception_model.get_layer('mixed7')
print(final_layer.output)
print(final_layer.output_shape)

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_282 (Conv2D)             (None, 74, 74, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_282 (BatchN (None, 74, 74, 32)   96          conv2d_282[0][0]                 
__________________________________________________________________________________________________
activation_282 (Activation)     (None, 74, 74, 32)   0           batch_normalization_282[0][0]    
_______________________________________________________________________________________

마지막 레이어인 output layer(Dense)가 없죠? 지금까지 Conv+Pooling으로 이미지 해체분석을 해줬습니다.  
그럼 이제 레이어를 연결해 봅시다

In [14]:
import tensorflow as tf

layer1=tf.keras.layers.Flatten()(final_layer.output)
later2=tf.keras.layers.Dense(1024,activation='relu')(layer1)
drop1=tf.keras.layers.Dropout(0.2)(later2)
layer3=tf.keras.layers.Dense(1,activation='sigmoid')(drop1)

model=tf.keras.Model(inception_model.input,layer3)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(train_ds,validation_data=val_ds,epochs=2)

Epoch 1/2
313/313 [==============================] - 964s 3s/step - loss: 0.3495 - acc: 0.9262 - val_loss: 0.0910 - val_acc: 0.9636
Epoch 2/2
313/313 [==============================] - 954s 3s/step - loss: 0.0432 - acc: 0.9836 - val_loss: 0.0942 - val_acc: 0.9696


이렇게 하면 위에 fine tunning에서 trainable 한 값들이 아주 조금씩, weight들이 업데이트 됩니다. pre train 된 layer 모델이랑 weight를 구할수있으면 여기서부터 구하는게 요즘 유행하는 딥러닝 학습 방식입니다.

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(Ir=0.0001),metrics=['acc'])
model.fit(train_ds,validation_data=val_ds,epochs=2)